## Laden der Trainings- und Testdaten

In diesem Schritt laden wir die Trainings- und Testdatensätze aus den vorbereiteten CSV-Dateien. Dies ist eine wichtige Phase, um die Daten für das Modelltraining und die Evaluation vorzubereiten.

### Laden der Daten

Zunächst laden wir die Trainings- und Testdatensätze mit `pandas`:


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import cross_val_score


train_data= pd.read_csv('../0_DataPreparation/imputed/imputed_data_train_with_wettercode.csv')
test_data= pd.read_csv('../0_DataPreparation/imputed/imputed_data_test_with_wettercode.csv')

print(test_data.columns)
print(train_data.columns)



Index(['id', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Wochentag',
       'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'gefühl_Kalt', 'gefühl_Mild',
       'gefühl_Warm', 'Feiertag', 'KielLauf', 'Kieler_Triathlon', 'Fußball',
       'PaycheckEffect', 'InflationSensitivity', 'Sensitivity_High',
       'Sensitivity_Moderate', 'Windkategorie', 'Wind_Nicht windig',
       'Wind_Sehr windig', 'Wind_Windig', 'Jahreszeit_Frühling',
       'Jahreszeit_Herbst', 'Jahreszeit_Sommer', 'Jahreszeit_Winter', 'Ferien',
       'Tag_Kategorie', 'Wochenende', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv', 'Montag', 'Dienstag', 'Mittwoch',
       'Donnerstag', 'Freitag', 'Samstag', 'Sonntag', 'Brot', 'Brötchen',
       'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot'],
      dtype='object')
Index(['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur',
      

## Definieren der Features und des Ziels

In diesem Schritt werden die Merkmale (Features) und das Ziel (Target) für das Modell definiert. Das Ziel ist der "Umsatz", während die Features andere numerische Spalten sind.

### Features und Ziel

Zuerst definieren wir die Spalten, die wir als Features (Eingabedaten) verwenden möchten, und die Spalten, die wir vom Modell ausschließen müssen:


In [4]:

# Features und Ziel definieren
# Beispiel: Ziel ist "Umsatz", Features sind andere numerische Spalten
feature_to_drop= ['id','Umsatz', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode',  'Wochentag',
       'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'InflationSensitivity',  'Windkategorie',  
       'Tag_Kategorie',  'Inflation_Kategorisierung',
       'Montag', 'Dienstag', 'Mittwoch',
       'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
feature_to_drop_y= ['id', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode',  'Wochentag',
       'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'InflationSensitivity',  'Windkategorie',  
       'Tag_Kategorie',  'Inflation_Kategorisierung',
       'Montag', 'Dienstag', 'Mittwoch',
       'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']


X_train = train_data.drop(columns=feature_to_drop)  # Features: Alle außer 'Umsatz' und 'Datum'

y_train = train_data['Umsatz']  # Ziel: Umsatz

X_test= test_data.drop(columns=feature_to_drop_y)




# Train-Test-Split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineares Regressionsmodell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


## Hinzufügen der Vorhersagen zu den Testdaten und Erstellen der Ergebnistabelle

In diesem Schritt fügen wir die Vorhersagen des Modells (`y_pred_test`) als neue Spalte zu den Testdaten hinzu und erstellen eine Tabelle mit den relevanten Informationen für die Einreichung.

### Vorhersagen zu den Testdaten hinzufügen

Wir fügen die Vorhersagen als neue Spalte `Umsatz` in die Testdaten ein und erstellen eine neue Tabelle:


In [5]:
# Add y_pred_test as a new column in X_test
X_test_with_predictions = test_data.copy()  # Make a copy to avoid modifying the original data
X_test_with_predictions['Umsatz'] = y_pred_test

# Select the 'id' and 'Umsatz' columns for the final table
result_table = X_test_with_predictions[['id', 'Umsatz']]

submission_sample = pd.read_csv('../0_DataPreparation/kaggel/sample_submission.csv')


kaggel_result = pd.merge(submission_sample, result_table, on=['id'], how='left').fillna(0)

kaggel_result=kaggel_result.drop(columns=['Umsatz_x'])
kaggel_result.rename(columns={'Umsatz_y': 'Umsatz'}, inplace=True)

kaggel_result.to_csv('submission.csv',index=False)
result_table.to_csv('submission_without_nan.csv',index=False)

## Modellbewertung und Analyse der Koeffizienten

Nachdem das Modell trainiert wurde, bewerten wir seine Leistung anhand verschiedener Metriken und analysieren die wichtigsten Koeffizienten.

### Vorhersagen und Zielwerte

Wir drucken die Vorhersagen für die Testdaten (`y_pred_test`) und berechnen dann verschiedene Metriken, um die Leistung des Modells zu bewerten.


In [6]:
# Modellbewertung
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# Ergebnisse ausgeben
print(f"Trainingsdaten: MSE = {mse_train:.2f}, R² = {r2_train:.2f}")
print("Anzahl der Features:", len(X_train.columns))
print("Anzahl der Koeffizienten:", len(model.coef_))
# Wichtige Koeffizienten und Features
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': model.coef_})
print("\nKoeffizienten der Features:")
print(coefficients.sort_values(by='Coefficient', ascending=False))

scores = cross_val_score(model, X_train, y_train, cv=30, scoring='r2')
print(scores)

Trainingsdaten: MSE = 5499.21, R² = 0.74
Anzahl der Features: 28
Anzahl der Koeffizienten: 28

Koeffizienten der Features:
                              Feature   Coefficient
13                        Wind_Windig  1.251513e+15
11                  Wind_Nicht windig  1.251513e+15
12                   Wind_Sehr windig  1.251513e+15
10               Sensitivity_Moderate  5.136555e+14
16                  Jahreszeit_Sommer  2.309259e+14
15                  Jahreszeit_Herbst  2.309259e+14
14                Jahreszeit_Frühling  2.309259e+14
17                  Jahreszeit_Winter  2.309259e+14
23                           Brötchen  2.004791e+14
22                               Brot  2.004791e+14
27                         Saisonbrot  2.004791e+14
9                    Sensitivity_High  1.133408e+11
4                            Feiertag  6.691098e+01
19                         Wochenende  5.313209e+01
18                             Ferien  3.879072e+01
20  Inflation_Kategorisierung_Neutral  3.0319